In [23]:
# !pip install simpletransformers

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
cd drive/MyDrive/Colab\ Notebooks/NER-BERT/

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/NER-BERT/'
/content/drive/.shortcut-targets-by-id/1QvIgM-ibjcwqQjycNIevg9-JTpfvTupz/NER-BERT


In [26]:
ls

 cache_dir/                         NER-DistilBERT.ipynb
'Check ERROR_1.csv'                 NER_finetuning_BERT.ipynb
'Check ERROR_correct_set.csv'       NER_other_models.ipynb
'Check ERROR.csv'                   NER_topic4.ipynb
'Clean Dataset_del_nan.ipynb'       outputs/
'Clean Dataset.ipynb'               reduced_ner_corpus_32.csv
'Clean Dataset_ver2.ipynb'          reduced_ner_corpus_512.csv
 corpus_for_ner_modified_ver2.csv   reduced_ner_corpus_8.csv
 model_32_bert.pth                  result_status.txt
 model_32_distilbert.pth            runs/
 NER-BERT.ipynb                     test.ipynb
'NER- biomed_roberta_base.ipynb'    topic4_dataset.csv


In [27]:
import time
start_time = time.time()

In [28]:
import pandas as pd
data = pd.read_csv("reduced_ner_corpus_512.csv",encoding="latin1" )

In [29]:
data = data.dropna()

In [30]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [31]:
data["sent_number"] = LabelEncoder().fit_transform(data["sent_number"] )

In [32]:
data.rename(columns={"sent_number":"sentence_id","sent_tokens":"words","flag":"labels"}, inplace =True)

In [33]:
data["labels"] = data["labels"].str.upper()

In [34]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [35]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [36]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

### **Model Training**

- https://simpletransformers.ai/docs/ner-specifics/
- https://simpletransformers.ai/docs/ner-model/
- https://huggingface.co/docs/transformers/model_doc/bert

*   Supported Model types: https://simpletransformers.ai/docs/ner-specifics/
*   Specific Models: https://huggingface.co/models



Interesting Models: 


*   https://huggingface.co/dmis-lab/biobert-v1.1
*   https://huggingface.co/microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext
* https://huggingface.co/allenai/biomed_roberta_base
* https://huggingface.co/michiyasunaga/BioLinkBERT-base


In [37]:
from simpletransformers.ner import NERModel,NERArgs

In [38]:
import torch

In [39]:
label = data["labels"].unique().tolist()
print(label)

['O', "'B-TISSUE'", "'B-ORG'", "'B-CHEMICAL'", "'B-CELL_COMPONENT'", "'B-CORONAVIRUS'", "'B-GPE'", "'I-GPE'", "'B-GROUP'", "'B-DISEASE_OR_SYNDROME'", "'B-GENE_OR_GENOME'", "'I-GENE_OR_GENOME'", "'I-CHEMICAL'", "'B-ORDINAL'", "'I-DISEASE_OR_SYNDROME'", "'I-GROUP'", "'B-PERSON'", "'I-PERSON'", "'I-ORG'", "'B-VIRUS'", "'B-SIGN_OR_SYMPTOM'", "'B-SUBSTRATE'", "'B-BODY_PART_ORGAN_OR_ORGAN_COMPONENT'", "'I-ORDINAL'", "'B-ORGANISM'", "'B-SOCIAL_BEHAVIOR'", "'B-WILDLIFE'", "'B-CELL'", "'I-CELL_COMPONENT'", "'I-ORGANISM'", "'B-EUKARYOTE'", "'B-LOC'", "'B-FOOD'", "'I-CELL'", "'B-CELL_FUNCTION'", "'B-BODY_SUBSTANCE'", "'B-ANATOMICAL_STRUCTURE'", "'B-LABORATORY_PROCEDURE'", "'B-LABORATORY_OR_TEST_RESULT'", "'I-TISSUE'", "'I-LABORATORY_PROCEDURE'", "'B-IMMUNE_RESPONSE'", "'I-LABORATORY_OR_TEST_RESULT'", "'I-EUKARYOTE'", "'I-BODY_PART_ORGAN_OR_ORGAN_COMPONENT'", "'B-LIVESTOCK'", "'I-ANATOMICAL_STRUCTURE'", "'I-VIRUS'", "'B-MATERIAL'", "'I-LOC'", "'I-CELL_FUNCTION'", "'I-SIGN_OR_SYMPTOM'"]


In [40]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32


In [41]:
# allenai/biomed_roberta_base

model = NERModel('roberta', 'allenai/biomed_roberta_base',labels=label,args =args, use_cuda = False)

Some weights of the model checkpoint at allenai/biomed_roberta_base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at allenai/biomed_roberta_base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on

In [42]:
# dmis-lab/biobert-v1.1

model = NERModel('bert', 'dmis-lab/biobert-v1.1',labels=label,args =args, use_cuda = False)

Downloading:   0%|          | 0.00/462 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

In [43]:
# microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext

model = NERModel('bert', 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext',labels=label,args =args, use_cuda = False)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForToken

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [44]:
# microsoft/michiyasunaga/BioLinkBERT-base

model = NERModel('bert', 'michiyasunaga/BioLinkBERT-base',labels=label,args =args, use_cuda = False)

Downloading:   0%|          | 0.00/559 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at michiyasunaga/BioLinkBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/220k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/379 [00:00<?, ?B/s]